In [2]:
import pandas as pd
import numpy as np

train = pd.read_csv("../data/train.csv")

In [3]:
interactions = (
    train.groupby(["msno", "song_id"])
         .size()
         .reset_index(name="play_count")
)

In [5]:
user_codes = interactions["msno"].astype("category").cat.codes
item_codes = interactions["song_id"].astype("category").cat.codes

user_id_map = dict(enumerate(interactions["msno"].astype("category").cat.categories))
item_id_map = dict(enumerate(interactions["song_id"].astype("category").cat.categories))


In [6]:
user_mapping = dict(enumerate(interactions["msno"].astype("category").cat.categories))
item_mapping = dict(enumerate(interactions["song_id"].astype("category").cat.categories))

In [6]:
from scipy.sparse import csr_matrix

user_item_matrix = csr_matrix(
    (interactions["play_count"],
     (user_codes, item_codes))
)

In [9]:
user_item_matrix.shape

(30755, 359966)

In [7]:
from implicit.als import AlternatingLeastSquares

als_model = AlternatingLeastSquares(
    factors=64,
    regularization=0.1,
    iterations=20,
    random_state=42
)

als_model.fit(user_item_matrix)


/Users/aditisonkar12/Desktop/Python Projects/Music-recommendation-system/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 20/20 [00:46<00:00,  2.33s/it]


In [12]:
als_model.fit(user_item_matrix.T)

/Users/aditisonkar12/Desktop/Python Projects/Music-recommendation-system/venv/lib/python3.10/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed csc_matrix instead. Converting to CSR took 0.04389619827270508 seconds
  warnings.warn(
100%|██████████| 20/20 [00:46<00:00,  2.30s/it]


In [8]:
user_id = 0
user_items = user_item_matrix[user_id]

ids, scores = als_model.recommend(
    userid=user_id,
    user_items=user_items,
    N=10
)

ids, scores


(array([ 31486, 136455, 169515, 342271,  75292, 230572, 149281, 215054,
        225460, 243500], dtype=int32),
 array([0.72222495, 0.67358327, 0.61404586, 0.6048321 , 0.59560156,
        0.57939214, 0.56619847, 0.5528924 , 0.54900277, 0.52264076],
       dtype=float32))